# DIY ZHL

## References etc.

  * Mark Powell's "Deco for Divers" has all the background information in a succinct and easy to read form. The book should be available on interlibrary loan, ask your librarian.
  
  * Erik Baker's papers: "Decolessons", "Understanding M-values", "Clearing up the confusion about Deep Stops", are more technical and tend to omit much of the background info. PDFs are freely available on-line from several sources.
  
  * Buhlmann's "Decompression - Decompression sickness".
  
  * Several open-source implementations are out there, most notable Subsurface software, but you have to read C.
  
  * Plenty of other on-line sources, e.g. Stuart Morrison's "DIY Decompresion"


## Primer

### Dissolved gas

Gas is driven in and out of tissue by the difference in pressure: "outside" aka ambient $ P_{amb} $ &nbsp; and "inside" aka dissolved gas pressure, $ P_{t} $. The rate is assumed to be $ ln(2) $

**Partial pressure** refers to pressures of individual gases in the breathing mix: oxygen, nitrogen, etc.

### Tissue compartments

To simulate real tissue the model uses hypothetical "compartments". They are defined by the time $ T_{ h } $ it takes for the pressure difference (above) to get to -- since we're using log 2 -- half (or double, depending on whether the tissues in on- or off-gassing) its intial value. That time is callled ``half time``. It is usually given in minutes.

The body is modelled as a set of "tissue compartments" with different half-times. They have nothing to do with real tissues, but to illustrate the concept it is usually said that blood is "fast": gas gets in and out very quickly, whereas bone is very "slow".

### M-values

The greater the difference between $ P_{amb} $ &nbsp; and $ P_t $, the faster the gas flows. There is, however, such thing as too fast. To recreational divers "too fast" is only relevant on the way up.

The model defines "maximum allowed pressure difference" value, ``M-value`` for short. The model does not attempt to quantify the risk, it only works in binary "above the M-value" (bad) or "below the M-value" (OK) terms.

Haldane originally set his M-value as a fixed ratio (2:1). Workman replaced it with a function: a straight line $ M = \Delta M + M_0 $ with greater overpressure tolerated at greater depths.

## Water vapour etc. in alveolae

Breathing gas in respiratory passages is diluted with water vapour and expelled carbon dioxide. The fraction of water vapour and carbon dioxide is subtracted from the partial pressure of inert gas.

$
\begin{align}
P_{alv} = ( P_{amb} - P_{H_{2}O} + ( 1 - RQ ) / RQ * P_{CO_{2}} ) Q
\end{align}
$

where
  * $ P_{ CO_2 } $ is pressure of carbon dioxide, 40 mm Hg or 0.0534 bar,
  * $ RQ $ is respiratory quotient,
  * $ P_{ H_{2}O } $ is pressure of water vapour, 47 mm Hg or 0.0627 bar,
  * $ Q $ is the fraction of inert gas,
  * $ P_{amb} $ is ambient pressure.

$ RQ $ is a ratio of oxygen consumed to carbon dioxide produced in metabolism, it depends on nutrition and exertion and is in the range 0.7..1. Smaller number results in greater calculated gas loading. Buhlmann uses 1 whereas Shreiner uses 0.8 and USN uses 0.9. 

## Schreiner equation to calculate inert gas pressure in a tissue compartment


$
\begin{align}
P_{t}(t) = P_{alv0} + R * (t - 1 / k) - (P_{alv0} - P_{t0} - R / k) * e^{-k * t}
\end{align}
$

*where*

  * $ P_{t}(t) $ : partial pressure of inert gas in the tissue (bar) at time $ t $
  
  * $ P_{alv0} $ : initial alveolar partial pressure of the gas in the breathing mix (bar)
  
  * $ P_{t0} $ : initial partial pressure of the gas in the tissue (bar) (at time $ t0 $)
  
  * $ R $ is rate of change in inert gas pressure in the breathing mix, bar/min. This is rate of change in ambient pressure, $ \Delta P $ times fraction of inert gas. $ \Delta P $ is negative for ascent, positive for descent, zero for constant depth.

  * $ t $ : time of exposure in minutes

  * $ k $ : gas decay constant for a tissue compartment: $ k = \frac  {\ln(2)} {T_{h}} $

    * $ \quad T_{h} $ : gas "half-life" time for tissue compartment in minutes

  * $ e $ : 2.7182818284590452353602874713527 and counting

Note that for constant depth this reduces to a "short form" given in many sources, the code below just uses the full form with $ R = 0 $

## M-values

### Workman

Workman-style M-value $ M $ is defined in terms of $ \Delta M $ and $ M_{ 0 } $, where

  * $ M_{0} $ is the surfacing M-value and
  * $ \Delta M $ is the slope of M-value line: multiply by depth to get tha actual number at that depth.

$ M = \Delta M * Depth + M_{0} $


$ Tolerated Depth = ( P - M_{0} ) / \Delta M $

where Depth is measured as pressure from sea level and Tolerated Depth is how high you can ascend without exceeding the M-value.

### Buhlmann

In Buhlmann model the M-values are expresed in terms of two coeficients: $ a $ and $ b $. In addition, it is using absolute instead od sea-level pressure for diving at altitude, and comes with separate numbers for different inert gases. Buhlmann's model calls "Tolerated Depth" an "ascent ceiling".

### Conversion

  * Workman to Buhlmann:
  
   * $ a = M_{0} - \Delta M * P_{ \text{ amb. (surface at sea level) }} $ 
   
   * $ b = 1 / \Delta M $  

  * Buhlmann to Workman:

   * $ \Delta M = 1 / b $
   
   * $ M_{0} = a + \frac { P_{ \text{ amb. (surface at sea level) }}} { b } $
   
**Note** that this only really works with $ \Delta M $ greater than 1. See Erik Baker's "M-values" for the why.
   

## No-stop time

### by (variant of) Schreiner equation

This needs Workman-style M-values:

$
\begin{align}
P_{no-deco} \ = ( M_{0} - P_{alv0} - R * (t_{asc} - 1/k) ) * e ^ {k * t_{asc}} \ + P_{alv0} - R / k
\end{align}
$
  * $ P_{no-deco} $ &nbsp; is the pressure at which ascent must be started (bar),
  * $ M_{0} $ is the surfacing M-value,
  * $ P_{alv0} $ is alveolar partial pressure (bar),
  * $ t_{asc} $ is the time needed for ascent (minutes): depth divided by speed.
  * $ R, k, e $ are the same as in Schreiner's equation above.
  
You have to figure out how long it'll take to get from the current inspired pressure $ P_{t} $ to $ P_{no-deco} $ &nbsp; : run `schreiner()` in a loop adding a minute of exposure on each iteration until you hit $ P_{no-deco} $ &nbsp; (or do a fancy binary search from, say, 99 minutes down, etc.)

### by Buhlmann equation

Buhlmann equation calculates Tolerated Depth in absolute pressure and calls it "safe ascent ceiling":

$
\begin{align}
P_{amb.tol.} = ( P_{t} - a ) * b
\end{align}
$

where
  * $ P_{amb.tol.} $ &nbsp; is the ascent ceiling,
  * $ P_{t} $ is current inert gas pressure in the tisue, and
  * $ a $ and $ b $ are Buhlmann's coefficients.

When the calculated $ P_{amb.tol.} $ &nbsp; is above 1, i.e. under the water surface (assuming surface pressure is 1 bar of course), that means we have a deco ceiling and must stop and off-gas there.

Buhlmann equation modified by Erik Baker's Gradient Factors is

$
\begin{align}
P_{amb.tol.} = ( P_{t} - a * GF ) / ( GF / b + 1.0 - GF )
\end{align}
$

where $ GF $ is the "current" gradient factor. 

(Without the gradient factor: $ GF = 1 $, that reduces to $ ( P_{t} - a * 1 ) / ( 1/b + 1 - 1 ) => ( P_{t} - a ) / ( 1/b ) =>  ( P_{t} - a ) * b $: Buhlmann's original.)

Gradient Factors plural are a pair of numbers: $ GFLo $ that applies at first decompression stop and $ GFHi $ that applies at the last stop: the surface. The "current" $ GF $ at any given depth between those two points is calculated from a simple proportion. **Note** that this only works when you have a relatively deep decompression stop.

### You can calculate no-stop time
by running `shreiner()` and `buhlmann()` in a loop, adding a minute of exposure at each iteration, until the latter produces a ceiling. **Note** however, that neither this method nor the equation above factor in off-gassing that will take place during ascent.

### Helium

With multiple inert gases, each has to be tracked separately by Schreiner equation.

For Buhlmann's formula you add up the partial pressures and proportionally adjust the $ a $ and $ b $:

$ P_{t} = P_{iN_2} + P_{iHe} $

$ a = (a_{N_2} * P_{N_2} + a_{He} * P_{He}) / P_{t} $

$ b = (b_{N_2} * P_{N_2} + b_{He} * P_{He}) / P_{t} $



## Decompression stop

... is ascent ceiling rounded to next 3 metres down.

The best I could find, in Buhlmann's "Decompression", is in open water it's hard to maintain stops at smaller intervals. Buhlmann suggests using smaller intervals or even "ride the M-value line" continuously for in-chamber decompression there. (**Note** this assumes no gas switching.)

Once you're at deco stop, run `schreiner()` and `buhlmann()` in a loop adding a minute on each iteration, as usual, until your next ceiling is at your next stop: 3 metres above you current depth if using the 3 m spacing.

## Tissue compartments

Tables are coded as python dicts: keys for each compartment are ``t``, ``a``, and ``b`` for half-time and the coefficients resp. E.g. ZH-L12's $ b $ coefficient for nitrogen in TC 3 is ``ZHL12N[3]["b"]``


### ZH-L16

**Note** that you have two options for the 1st (fastest) tissue compartment: 4 minutes or 5. Pick one. The 5-minute TC is usually called "1b" and is keyed as "1.1" in the table below.

The ''a'' key in the nitrogen tables is a dict itself with three sub-keys: ''A'', ''B'', adn ''C'' for the three versions of ZH-L16 resp. Helium numbers seem to be unobtainable for ''A'' or ''C'', nor for the 4-minute tissue compartment. I.e. if you want to use this for trimix, ZH-L16B with 5-minute TC is your only option.

M-values for "-A" were calculated as

$
\begin{align}
a = 2 * T_{h} ^ {- \frac{1}{3}}
\end{align}
$

$
\begin{align}
b = 1.005 - T_{h} ^ {- \frac{1}{2}}
\end{align}
$


  * $ T_{h} $ is tissue compartment half-time


### Workman

Not useful as is, would need to be recalculated for $ \Delta M $ instead of $ M $ first, then converted to Buhlmann's `a` and `b` coefficients.

### DSAT

Convert to Buhlmann's `a` and `b` using $ \Delta M = 1 $

# No-Stop Time aka NDL

From **Deco for Divers**:

\begin{align}
P_{no-stop} = (M_{0} - P_{alv0} - R (t_{asc} - 1/k)) e^{k * t_{asc}} + P_{alv0} - R/k
\end{align}

where

  * $ M_{0} $ : M-value at sea level (bar)
  * $ t_{asc} $ : time needed for ascent, depth/speed (min)
  * $ P_{no_stop} $ : partial pressure at which to start the ascent (bar)
  * $ P_{alv0} $ : alveolar partial pressure
  * $ d $ : depth (m)
  * $ k $ : ln(2)/tissue half-time
  * $ R $ : is the rate of change in inert gas paressure, $ \Delta P $ times fraction of inert gas (bar/min)

This gives no-stop pressure $ P_{no-stop} $ at a given depth (through $ P_{alv0} $) but not the time $ t $ when you're going to reach it.

From Baker's **No-Stop Time**:

\begin{align}
t = (-1/k) * ln( (P_{alv0} - M_{0})/(P_{alv0} - P_{tc}) )
\end{align}

-- as above but also accounts for

  * $ P_{tc} $ : current dissolved gas pressure in tissue compartment
  
The problem with Baker's version is it has to avoid division by zero and a logarithm of negative number, or of number greater than 1. So it only works **iff** $ P_{alv0} > M_{0} > P_{tc} $ 

**So** you have to exclude tissue compartments whose $ M_{0} $ is less than current $ P_{alv0} $

## Code

Run the cell below to load python code (assuming you have the `diyzhl.py` in the same directory as the notebook file).

The functions are written to closely follow the above formulae. That includes calculating constants on the fly, like e.g. $ K $ or $ P_{alv} $ &nbsp; -- you'd normally use pre-calculated numbers for your chosen parameters.

About the only added features are 
 * some of them save intermediate results in variables: mainly because I wanted to play with rounding at different steps. Note that although I round the numbers to 4 decimal digits, they're probably only meaningful to 2 at best.
 
 * Typecasts to `float` are there so that passing in not-a-number will throw an exception early.      
 
 * Similarly, `asserts` are used to catch division by zero early.